<a href="https://colab.research.google.com/github/Reno20/T5-summarizer-training/blob/main/T5%20news%20summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install evaluate
!pip install rouge_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 81.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import evaluate
import numpy as np
import pprint
import shutil

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

from datasets import load_dataset


In [ ]:
pp = pprint.PrettyPrinter()

data set preparation

In [ ]:
dataset = load_dataset("csv",data_files="bbc-news-summary.csv",split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
full_dataset = dataset.train_test_split(test_size=0.2 , shuffle=True)

In [ ]:
dataset_train = full_dataset["train"]
dataset_test = full_dataset["test"]


In [ ]:
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['File_path', 'Articles', 'Summaries'],
    num_rows: 1779
})
Dataset({
    features: ['File_path', 'Articles', 'Summaries'],
    num_rows: 445
})


In [ ]:
def find_longest_length(dataset):

  max_length=0
  counter_4k=0
  counter_2k=0
  counter_1k=0
  counter_500=0

  for text in dataset:
    corpus= [ word for word in text.split()]

    if len(corpus) > 4000:
      counter_4k+=1
    if len(corpus) > 2000:
      counter_2k+=1
    if len(corpus) > 1000:
      counter_1k+=1
    if len(corpus) > 500:
      counter_500+=1
    if len(corpus) > max_length:
      max_length = len(corpus)
  return max_length,counter_4k,counter_2k,counter_1k,counter_500

longest_article,counter_4k,counter_2k,counter_1k,counter_500 = find_longest_length(dataset_train["Articles"])
print("longest article:",longest_article)
print("article > 4k:",counter_4k)
print("article > 2k:",counter_2k)
print("article > 1k:",counter_1k)
print("article > 500:",counter_500)
longest_summary,counter_4k,counter_2k,counter_1k,counter_500 = find_longest_length(dataset_train["Summaries"])
print("longest summary:",longest_summary)
print("summary > 4k:",counter_4k)
print("summary > 2k:",counter_2k)
print("summary > 1k:",counter_1k)
print("summary > 500:",counter_500)

longest article: 4377
article > 4k: 1
article > 2k: 5
article > 1k: 17
article > 500: 347
longest summary: 2073
summary > 4k: 0
summary > 2k: 1
summary > 1k: 5
summary > 500: 13


In [ ]:
def find_avg_length(dataset):
  sentence_length=[]

  for text in dataset:
    corpus = [ word for word in text.split()]

    sentence_length.append(len(corpus))
  return np.mean(sentence_length)

avg_article = find_avg_length(dataset_train["Articles"])
print("avg article:",avg_article)
avg_summary = find_avg_length(dataset_train["Summaries"])
print("avg summary:",avg_summary)

avg article: 377.73412029229905
avg summary: 164.37549184935358


Configurations

In [ ]:
MODEL ='t5-base'
BATCH_SIZE = 1
NUM_PROCS = 4
EPOCHS = 10
MAX_LENGTH = 256
OUT_DIR = "/content/results_t5base"  # Default: Saves in Colab
#OUT_DIR = "/content/drive/MyDrive/results_t5small"  # to save in Google Drive

TOKENIZATION

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
#function to convert data into model inputs and targets
def preprocess_function(examples):
  inputs = [f"summarize: {article}" for article in examples["Articles"]]
  model_inputs = tokenizer(inputs, max_length=MAX_LENGTH, truncation=True , padding="max_length")

  #set up the tokenizer for targets
  targets = [summary for summary in examples["Summaries"]]
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(
        targets, max_length=MAX_LENGTH, truncation= True , padding="max_length"
    )
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

#applying function to whole dataset
tokenized_train = dataset_train.map(preprocess_function,batched=True, num_proc=NUM_PROCS)
tokenized_test = dataset_test.map(preprocess_function,batched=True, num_proc= NUM_PROCS)

Map (num_proc=4):   0%|          | 0/1779 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=4):   0%|          | 0/445 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

MODEL

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
#total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters count: {total_params:,}")
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters count: {trainable_params:,}")

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Total parameters count: 222,903,552
Trainable parameters count: 222,903,552


ROUGE Metric

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0] ,eval_pred.label_ids
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=['rouge1','rouge2','rougeL']
    )

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:

    training_args = TrainingArguments(
        output_dir=OUT_DIR,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        dataloader_num_workers=0,
        dataloader_pin_memory=False,
        gradient_accumulation_steps=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=OUT_DIR,
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=200,
        save_strategy="epoch",
        save_total_limit=2,
        report_to="tensorboard",
        learning_rate=0.0001,
        bf16=True,

    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        preprocess_logits_for_metrics=preprocess_logits_for_metrics,
        compute_metrics=compute_metrics
    )

history = trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
200,1.207200,1.304255,0.754600,0.603400,0.704300,200.510100
400,1.380100,1.207085,0.761200,0.611500,0.714300,200.501100
600,1.182500,1.169898,0.766000,0.615300,0.718000,200.501100
800,1.087700,1.152461,0.767500,0.617400,0.719700,200.501100
1000,1.014400,1.142267,0.768400,0.616900,0.720200,200.501100
1200,0.947800,1.137721,0.770000,0.618600,0.721800,200.501100
1400,0.889900,1.138069,0.771200,0.619600,0.722200,200.501100
1600,0.955400,1.140394,0.772000,0.621500,0.723400,200.501100
1800,0.920700,1.132805,0.772400,0.622000,0.723900,200.501100
2000,0.956000,1.136529,0.772200,0.621800,0.723900,200.501100


In [ ]:
model.save_pretrained(OUT_DIR)
model.save_pretrained(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

#Zip the trained model for download
zip_filename = "trained_t5_model.zip"
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', OUT_DIR)

'/content/trained_t5_model.zip'

Inference]


In [ ]:
from transformers import T5ForConditionalGeneration , T5Tokenizer
import glob

In [ ]:
# Unzip the uploaded model
!unzip trained_t5_model.zip -d /content/

# Load the model and tokenizer
model_path = "/content/results_t5base"  # Change if unzipped elsewhere
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

Archive:  trained_t5_model.zip
replace /content/events.out.tfevents.1740463571.c04da69dbcf3.227.0? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/events.out.tfevents.1740463571.c04da69dbcf3.227.0  
replace /content/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/tokenizer_config.json  
replace /content/config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/config.json    
replace /content/model.safetensors? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/model.safetensors  

  inflating: /content/special_tokens_map.json  
  inflating: /content/added_tokens.json  
  inflating: /content/generation_config.json  
  inflating: /content/spiece.model   
  inflating: /content/checkpoint-2220/training_args.bin  
  inflating: /content/checkpoint-2220/config.json  
  inflating: /content/checkpoint-2220/model.safetensors  
  inflating: /content/checkpoint-2220/rng_state.pth  
  inflating: /content/checkpoint-2220

In [ ]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True
    )

    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=100,
        num_beams=num_beams,
        # early_stopping=True,
    )

    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
test_text = """Budget to set scene for election..Gordon Brown will seek to put the economy at the centre of Labour's bid for a third term in power when he delivers his ninth Budget at 1230 GMT. He is expected to stress the importance of continued economic stability, with low unemployment and interest rates. The chancellor is expected to freeze petrol duty and raise the stamp duty threshold from £60,000. But the Conservatives and Lib Dems insist voters face higher taxes and more means-testing under Labour...Treasury officials have said there will not be a pre-election giveaway, but Mr Brown is thought to have about £2bn to spare...- Increase in the stamp duty threshold from £60,000. - A freeze on petrol duty. - An extension of tax credit scheme for poorer families. - Possible help for pensioners The stamp duty threshold rise is intended to help first time buyers - a likely theme of all three of the main parties' general election manifestos. Ten years ago, buyers had a much greater chance of avoiding stamp duty, with close to half a million properties, in England and Wales alone, selling for less than £60,000. Since then, average UK property prices have more than doubled while the starting threshold for stamp duty has not increased. Tax credits As a result, the number of properties incurring stamp duty has rocketed as has the government's tax take. The Liberal Democrats unveiled their own proposals to raise the stamp duty threshold to £150,000 in February...The Tories are also thought likely to propose increased thresholds, with shadow chancellor Oliver Letwin branding stamp duty a "classic Labour stealth tax". The Tories say whatever the chancellor gives away will be clawed back in higher taxes if Labour is returned to power. Shadow Treasury chief secretary George Osborne said: "Everyone who looks at the British economy at the moment says there has been a sharp deterioration in the public finances, that there is a black hole," he said. "If Labour is elected there will be a very substantial tax increase in the Budget after the election, of the order of around £10bn."..But Mr Brown's former advisor Ed Balls, now a parliamentary hopeful, said an examination of Tory plans for the economy showed there would be a £35bn difference in investment by the end of the next parliament between the two main parties. He added: "I don't accept there is any need for any changes to the plans we have set out to meet our spending commitments."..For the Lib Dems David Laws said: "The chancellor will no doubt tell us today how wonderfully the economy is doing," he said. "But a lot of that is built on an increase in personal and consumer debt over the last few years - that makes the economy quite vulnerable potentially if interest rates ever do have to go up in a significant way." SNP leader Alex Salmond said his party would introduce a £2,000 grant for first time buyers, reduce corporation tax and introduce a citizens pension free from means testing. Plaid Cymru's economics spokesman Adam Price said he wanted help to get people on the housing ladder and an increase in the minimum wage to £5.60 an hour."""

summary = summarize_text(test_text, model, tokenizer)
print("Generated Summary:", summary)


Generated Summary: The chancellor is expected to freeze petrol duty and raise the stamp duty threshold from £60,000.The Tories are also thought likely to propose increased thresholds, with shadow chancellor Oliver Letwin branding stamp duty a "classic Labour stealth tax".Treasury officials have said there will not be a pre-election giveaway, but Mr Brown is thought to have about £2bn to spare."If Labour is elected there will be a very substantial tax


In [ ]:
from google.colab import files
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>